The work above was concluded as part of my final project of the Master in Data Science at Nuclio Digital School. This dataset is a case of study from the M5 competition in Kaggle about timeseries forecasting of item sales from Walmart. Although the competition aim to get the best RMSSE score, we are goingo to look at this notebook the following way:

- Exploratory Data Analysis (EDA)
- Model train using DARTS
- Build an API of stock replenishment using the trained model

# Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import joblib

from darts import TimeSeries
from darts.models import NBEATSModel
from darts.dataprocessing.transformers import Scaler
from darts.metrics import rmse, mae

In [5]:
#Prices data set
file_id_1 = "1iutsbqnuMuUgmd_FuKt_1x5s0whbdV2U"
download_url_1 = f"https://drive.google.com/uc?id={file_id_1}"
item_price_1 = pd.read_csv(download_url_1)

file_id_2 = "17PbW-iHxFaYJor4tEzYopwBkMCi-8VRm"
download_url_2 = f"https://drive.google.com/uc?id={file_id_2}"
item_price_2 = pd.read_csv(download_url_2)

file_id_3 = "1Lz-QjwhM0YGTpTddQX2nyTEUBORrnsba"
download_url_3 = f"https://drive.google.com/uc?id={file_id_3}"
item_price_3 = pd.read_csv(download_url_3)

file_id_4 = "1MQBKY7sBZO1pe2TgSHa2pk1wxCOTkpIi"
download_url_4 = f"https://drive.google.com/uc?id={file_id_4}"
item_price_4 = pd.read_csv(download_url_4)

file_id_5 = "1m3NIwllKxkpZ3uwVHczXZy8tIvFAkxib"
download_url_5 = f"https://drive.google.com/uc?id={file_id_5}"
item_price_5 = pd.read_csv(download_url_5)

item_price = pd.concat([item_price_1, item_price_2, item_price_3, item_price_4, item_price_5], ignore_index=True)

In [7]:
#Events file
file_1 = "1X2XBvxf-DlRe1nym1wdL_Ct7GoCEtyLj"
url_1 = f"https://drive.google.com/uc?id={file_1}"
item_sales_1 = pd.read_csv(url_1)

file_2 = "1PIJsGPJCJm7gEOxChFNXjvb0E9Z4yb3F"
url_2 = f"https://drive.google.com/uc?id={file_2}"
item_sales_2 = pd.read_csv(url_2)

file_3 = "192CNuSqVj87xz5jVlK0K3rs4B4rTHkRQ"
url_3 = f"https://drive.google.com/uc?id={file_3}"
item_sales_3 = pd.read_csv(url_3)

file_4 = "1SFZoai5yzKSQn8s8EdxA16-FRgXGdLgy"
url_4 = f"https://drive.google.com/uc?id={file_4}"
item_sales_4 = pd.read_csv(url_4)

item_sales = pd.concat([item_sales_1, item_sales_2, item_sales_3, item_sales_4], ignore_index=True)

In [9]:
#Events file
file_id = "1gLVp1-FeQ1PbpfvnyVNS99FVwwr5ntj5"
download_url = f"https://drive.google.com/uc?id={file_id}"

events = pd.read_csv(download_url)

# Exploring datasets

## Price Dataset

In [11]:
item_price

,item,category,store_code,yearweek,sell_price
0,ACCESORIES_1_001,ACCESORIES,NYC_1,201328.0,12.7414
1,ACCESORIES_1_001,ACCESORIES,NYC_1,201329.0,12.7414
2,ACCESORIES_1_001,ACCESORIES,NYC_1,201330.0,10.9858
3,ACCESORIES_1_001,ACCESORIES,NYC_1,201331.0,10.9858
4,ACCESORIES_1_001,ACCESORIES,NYC_1,201332.0,10.9858
...,...,...,...,...,...
6965701,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965702,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965703,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000
6965704,SUPERMARKET_3_827,SUPERMARKET,PHI_3,NaN,1.2000


In [13]:
print(item_price.shape)
print(item_price.columns)

(6965706, 5)
Index(['item', 'category', 'store_code', 'yearweek', 'sell_price'], dtype='object')


In [15]:
item_price.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6965706 entries, 0 to 6965705
Data columns (total 5 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item        object 
 1   category    object 
 2   store_code  object 
 3   yearweek    float64
 4   sell_price  float64
dtypes: float64(2), object(3)
memory usage: 265.7+ MB


In [17]:
item_price.describe().T

,count,mean,std,min,25%,50%,75%,max
yearweek,6721786.0,201382.418346,145.018929,201105.000,201248.0000,201410.0,201515.000,201617.00
sell_price,6965706.0,5.518273,4.387861,0.012,2.6201,4.2,7.176,134.15


In [19]:
item_price.describe(include=['object']).T

,count,unique,top,freq
item,6965706,3049,SUPERMARKET_3_587,2870
category,6965706,3,SUPERMARKET,3239821
store_code,6965706,10,BOS_2,713960


In [20]:
item_price.isnull().sum()

item               0
category           0
store_code         0
yearweek      243920
sell_price         0
dtype: int64

In [22]:
raw_nan = (item_price['yearweek'].isnull().sum() / len(item_price))
percent_nan = "{:.2%}".format(raw_nan)
print('The percentage of null data in column yearweek is', percent_nan)

The percentage of null data in column yearweek is 3.50%


Since we have diferent sales price across the same product in the same store, we are going to calculate the mean prices for each product in each store.

In [13]:
mean_prices = pd.DataFrame(item_price.groupby(['item', 'store_code'])['sell_price'].mean().reset_index())

In [27]:
mean_prices

,item,store_code,sell_price
0,ACCESORIES_1_001,BOS_1,10.969366
1,ACCESORIES_1_001,BOS_2,10.997126
2,ACCESORIES_1_001,BOS_3,11.072293
3,ACCESORIES_1_001,NYC_1,11.019346
4,ACCESORIES_1_001,NYC_2,10.997126
...,...,...,...
30485,SUPERMARKET_3_827,NYC_3,1.200000
30486,SUPERMARKET_3_827,NYC_4,1.200000
30487,SUPERMARKET_3_827,PHI_1,1.200000
30488,SUPERMARKET_3_827,PHI_2,1.200000


## Sales Dataset

In [31]:
item_sales

,id,item,category,department,store,store_code,region,d_1,d_2,d_3,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,SUPERMARKET_3_823_PHI_3,SUPERMARKET_3_823,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,2,...,2,0,0,0,0,0,1,0,0,1
30486,SUPERMARKET_3_824_PHI_3,SUPERMARKET_3_824,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,SUPERMARKET_3_825_PHI_3,SUPERMARKET_3_825,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,6,0,...,2,1,0,2,0,1,0,0,1,0
30488,SUPERMARKET_3_826_PHI_3,SUPERMARKET_3_826,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [33]:
print(item_sales.shape)
print(item_sales.columns)

(30490, 1920)
Index(['id', 'item', 'category', 'department', 'store', 'store_code', 'region',
       'd_1', 'd_2', 'd_3',
       ...
       'd_1904', 'd_1905', 'd_1906', 'd_1907', 'd_1908', 'd_1909', 'd_1910',
       'd_1911', 'd_1912', 'd_1913'],
      dtype='object', length=1920)


In [35]:
item_sales.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1920 entries, id to d_1913
dtypes: int64(1913), object(7)
memory usage: 446.6+ MB


In [37]:
item_sales.select_dtypes(include=['object']).isnull().sum()

id            0
item          0
category      0
department    0
store         0
store_code    0
region        0
dtype: int64

In [39]:
filtered_columns = item_sales.select_dtypes(include=['number']).isnull().sum() >= 1

# Verificar se alguma coluna está com o valor nulo
if filtered_columns.any():
    print("At least one column has null values.")
else:
    print("No column has null values.")

No column has null values.


In [41]:
item_sales.describe().T

,count,mean,std,min,25%,50%,75%,max
d_1,30490.0,1.070220,5.126689,0.0,0.0,0.0,0.0,360.0
d_2,30490.0,1.041292,5.365468,0.0,0.0,0.0,0.0,436.0
d_3,30490.0,0.780026,3.667454,0.0,0.0,0.0,0.0,207.0
d_4,30490.0,0.833454,4.415141,0.0,0.0,0.0,0.0,323.0
d_5,30490.0,0.627944,3.379344,0.0,0.0,0.0,0.0,296.0
...,...,...,...,...,...,...,...,...
d_1909,30490.0,1.159167,2.876026,0.0,0.0,0.0,1.0,88.0
d_1910,30490.0,1.149000,2.950364,0.0,0.0,0.0,1.0,77.0
d_1911,30490.0,1.328862,3.358012,0.0,0.0,0.0,1.0,141.0
d_1912,30490.0,1.605838,4.089422,0.0,0.0,0.0,2.0,171.0


In [42]:
item_sales.describe(include=['object']).T

,count,unique,top,freq
id,30490,30490,ACCESORIES_1_001_NYC_1,1
item,30490,3049,ACCESORIES_1_001,10
category,30490,3,SUPERMARKET,14370
department,30490,7,SUPERMARKET_3,8230
store,30490,10,Greenwich_Village,3049
store_code,30490,10,NYC_1,3049
region,30490,3,New York,12196


## Events Dataset

In [46]:
events

,date,weekday,weekday_int,d,event
0,2011-01-29,Saturday,1,d_1,NaN
1,2011-01-30,Sunday,2,d_2,NaN
2,2011-01-31,Monday,3,d_3,NaN
3,2011-02-01,Tuesday,4,d_4,NaN
4,2011-02-02,Wednesday,5,d_5,NaN
...,...,...,...,...,...
1908,2016-04-20,Wednesday,5,d_1909,NaN
1909,2016-04-21,Thursday,6,d_1910,NaN
1910,2016-04-22,Friday,7,d_1911,NaN
1911,2016-04-23,Saturday,1,d_1912,NaN


In [48]:
print(events.shape)
print(events.columns)

(1913, 5)
Index(['date', 'weekday', 'weekday_int', 'd', 'event'], dtype='object')


In [50]:
events.select_dtypes(include=['number', 'object']).isnull().sum()

date              0
weekday           0
weekday_int       0
d                 0
event          1887
dtype: int64

In [52]:
events.describe(include=['object']).T

,count,unique,top,freq
date,1913,1913,2011-01-29,1
weekday,1913,7,Saturday,274
d,1913,1913,d_1,1
event,26,5,SuperBowl,6


In [15]:
#changing the name of column d to day so we can merge the file later
events.rename(columns={'d':'day'}, inplace=True)

## Merging the datasets

We have 3 different datasets and we need to join them all. We are going to proceed as following:

- First, melting the day variable (d_1, d_2, ...) columns of 'item_sales' to put then in order using the pandas melt function.
- Second, merging item_sales and item_prices using the item and store_code.
- Third, mergind the resulting Data Frame above with the events on 'd'.

In [17]:
sales_melted = item_sales.melt(id_vars=['id', 'item', 'category', 'department', 'store', 'store_code', 'region'], var_name='day', value_name='units_sold')

In [19]:
sales_melted.head(10)

,id,item,category,department,store,store_code,region,day,units_sold
0,ACCESORIES_1_001_NYC_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
1,ACCESORIES_1_002_NYC_1,ACCESORIES_1_002,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
2,ACCESORIES_1_003_NYC_1,ACCESORIES_1_003,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
3,ACCESORIES_1_004_NYC_1,ACCESORIES_1_004,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
4,ACCESORIES_1_005_NYC_1,ACCESORIES_1_005,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
5,ACCESORIES_1_006_NYC_1,ACCESORIES_1_006,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
6,ACCESORIES_1_007_NYC_1,ACCESORIES_1_007,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0
7,ACCESORIES_1_008_NYC_1,ACCESORIES_1_008,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,12
8,ACCESORIES_1_009_NYC_1,ACCESORIES_1_009,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,2
9,ACCESORIES_1_010_NYC_1,ACCESORIES_1_010,ACCESORIES,ACCESORIES_1,Greenwich_Village,NYC_1,New York,d_1,0


In [21]:
sales_prices = sales_melted.merge(mean_prices, how='left', on=['item', 'store_code'])

In [23]:
#creating the final dataframe
df = sales_prices.merge(events, how='left', on='day')

# Downcast

In [26]:
df.dtypes

id              object
item            object
category        object
department      object
store           object
store_code      object
region          object
day             object
units_sold       int64
sell_price     float64
date            object
weekday         object
weekday_int      int64
event           object
dtype: object

Now we are going to save some memory because it is always good practice and it save us time!

In [29]:
def memory_usage_mb(df):
    return df.memory_usage(deep=True).sum() / (1024 * 1024)

# Memória original
memory_before = memory_usage_mb(df)
print(f"Memória antes: {memory_before:.2f} MB")

Memória antes: 36619.88 MB


In [25]:
#object column
object_columns = df.select_dtypes(include='object').columns.tolist()
df[object_columns] = df[object_columns].astype('category')

In [27]:
#date time
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

In [29]:
#numerical columns
df = df.astype(
    {
    'units_sold':'int16',
    'sell_price':'float16',
    'weekday_int':'int8'
    }
)

In [31]:
# events encoding, if it is an event 1, else 0.
df['event'] = np.where(df['event'].isna(), 0, 1).astype('int8')

In [56]:
memory_after = memory_usage_mb(df)

reduction_percentage = 100 * (memory_before - memory_after) / memory_before

print(f"Memory after: {memory_after:.2f} MB")
print(f"Memory reduction: {reduction_percentage:.2f}%")

Memory after: 1561.40 MB
Memory reduction: 96.24%


# Preparing data for modeling

In [37]:
df_nbeats = df.groupby(['date','event','weekday_int'], as_index=False).agg({'units_sold': 'sum'})

/var/folders/qk/1hlkl_vs42x841fcykv090lw0000gn/T/ipykernel_1073/2691831878.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df = df.groupby(['date','event','weekday_int'], as_index=False).agg({'units_sold': 'sum'})


ValueError: Length of values (1913) does not match length of index (26782)

In [ ]:
series = TimeSeries.from_dataframe(df_nbeats, 'date', 'units_sold').astype(np.float32)

series_covariates = TimeSeries.from_dataframe(df_nbeats, time_col='date',
                                              value_cols=['weekday_int', 'event']).astype(np.float32)